# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></div><div class="lev1 toc-item"><a href="#Imports" data-toc-modified-id="Imports-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Imports</a></div><div class="lev1 toc-item"><a href="#Useful-Scripts" data-toc-modified-id="Useful-Scripts-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Useful Scripts</a></div><div class="lev1 toc-item"><a href="#Remove-NaNs-from-dmstack-csv-to-get-txt" data-toc-modified-id="Remove-NaNs-from-dmstack-csv-to-get-txt-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Remove NaNs from dmstack csv to get txt</a></div><div class="lev1 toc-item"><a href="#Create-final_text.txt-using-IMCAT" data-toc-modified-id="Create-final_text.txt-using-IMCAT-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Create final_text.txt using IMCAT</a></div><div class="lev1 toc-item"><a href="#Plot-gmsq-histogram-using-final_text.txt" data-toc-modified-id="Plot-gmsq-histogram-using-final_text.txt-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Plot gmsq histogram using final_text.txt</a></div><div class="lev1 toc-item"><a href="#Run-Script" data-toc-modified-id="Run-Script-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Run Script</a></div>

# Introduction
Date: Nov 7, 2019

final_text.txt is created by imcat program after merging four lsst files (m,m9,l,l9)
after cleaning.

**Usual Filtering**
```python
column ==> deblend_nChild == 0.0
ellipticity ==> ellip < 1.5
nans ==> if nans in cols_select, remomve row

where,
cols_select = ['base_SdssCentroid_x', 'base_SdssCentroid_y',
       'base_SdssCentroid_xSigma','base_SdssCentroid_ySigma',
       'ext_shapeHSM_HsmShapeRegauss_e1','ext_shapeHSM_HsmShapeRegauss_e2',
        'base_SdssShape_flux']
```

**Check effects of flags**
```python
calib_psfCandidate == 0.0 # usual flag

# check effect of other flags on number density in 0.5 < gmsq < 1.1
```

# Imports

In [1]:
import json, os,sys
import numpy as np
import pandas as pd
import seaborn as sns
sns.set(color_codes=True)

pd.set_option('display.max_columns',200)

import matplotlib.pyplot as plt
%matplotlib inline

# Useful Scripts

In [2]:
def show_method_attributes(obj, ncols=7,start=None, inside=None):
    """ Show all the attributes of a given method.
    Example:
    ========
    show_method_attributes(list)
     """
    lst = [elem for elem in dir(obj) if elem[0]!='_' ]
    lst = [elem for elem in lst 
           if elem not in 'os np pd sys time psycopg2'.split() ]

    if isinstance(start,str):
        lst = [elem for elem in lst if elem.startswith(start)]
        
    if isinstance(start,tuple) or isinstance(start,list):
        lst = [elem for elem in lst for start_elem in start
               if elem.startswith(start_elem)]
        
    if isinstance(inside,str):
        lst = [elem for elem in lst if inside in elem]
        
    if isinstance(inside,tuple) or isinstance(inside,list):
        lst = [elem for elem in lst for inside_elem in inside
               if inside_elem in elem]

    return pd.DataFrame(np.array_split(lst,ncols)).T.fillna('')

In [3]:
dict_flags = json.load( open('dict_flags.json'))
# dict_flags

dict_flags['0']

'calib_detected'

# Remove NaNs from dmstack csv to get txt

In [1]:
%%writefile b01_remove_nans_dmstack.py
# Author  : Bhishan Poudel
# Date    : July 5, 2019
# Update  : Nov 7, 2019

# Description:
#===============
# Remove nans from dmstack output csv files and
# do some filterings to give txt files.
#
# Command: py b01_remove_nans_dmstack.py [0]
#
# Estimated time: 2min 10 sec
#
#
# Input/Oputputs:
#=================
# inputs : ../data/dmstack_csv/*.csv  (100*4 csv files)
# outputs: for True : dmstack_txt/dmstack_txt_0T/*.txt 225MB
#          for False: dmstack_txt/dmstack_txt_0F/*.txt 225MB
#
# Filtering:
#============

# 1. column ==> deblend_nChild==0
# 2. flag ==> calib_psfCandidate==False **Read flag from json**
# 3. ellipticity  ==> e =  sqrt(e1^2 + e2^2) < 1.5
# 4. selection ==> choose only few columns
# 5. nans ==> remove nans from all selected columns
# 6. delimiter ==> change delimiter from space to tab for imcat
#
#

#
# Note: 
# When reading columns ext_shapeHSM_HsmShapeRegauss_e1 and e2
# we read them combinedly as g in IMCAT, so original
# reduced shear will be g = g/2.
#
#
# Tech note:
# lst_flag_nums_str = '[0,1]'
# lst_flag_nums = [0,1]
# flag_nums_str = '0_1'
# flag_nums_true_false_str = '0_1T' and '0_1F'
#
#
import pandas as pd
import numpy as np
import os,sys
import glob
import json

import multiprocessing
from multiprocessing import Process

# constants
RANGE = 100

# global variables
dict_flags_all = json.load(open('dict_flags.json'))

# arguments
lst_flag_nums_str = sys.argv[1] # [0], [0,1] etc
lst_flag_nums = eval(lst_flag_nums_str)
str_flag_nums = '_'.join([str(i) for i in lst_flag_nums]) # '0_1'

odir_txtT = 'dmstack_txt/' + 'dmstack_txt_'+ str_flag_nums + 'T'
odir_txtF = 'dmstack_txt/' + 'dmstack_txt_'+ str_flag_nums + 'F' 

# create output folder if not exist
if not os.path.isdir('dmstack_txt'):
    os.makedirs('dmstack_txt')
    
if not os.path.isdir(odir_txtT):
    os.makedirs(odir_txtT)
    
if not os.path.isdir(odir_txtF):
    os.makedirs(odir_txtF)

def remove_nans(ifile,file_number, flag_TF):
    """ Remove nans and filter data from dmstack output csv file.

    There are 90 flags col0 to col89
    col90 is id is first column 'id'

    There are 90 flags and 77 columns.
    We exclude first column 'flags' and have 76 columns
    In total there are 90 + 76 = 166 columns.

    Columns selected:
    1   :  calib_psfCandidate (for filtering only)
    94  :  deblend_nChild (for filtering only)
    90  :  id
    102 :  base_SdssCentroid_x
    103 :  base_SdssCentroid_y
    104 :  base_SdssCentroid_xSigma
    105 :  base_SdssCentroid_ySigma
    127 :  ext_shapeHSM_HsmShapeRegauss_e1
    128 :  ext_shapeHSM_HsmShapeRegauss_e2
    114 :  ext_shapeHSM_HsmShapeRegauss_sigma

    # Added later for radius calculation
    133: 'ext_shapeHSM_HsmSourceMoments_xx',
    134: 'ext_shapeHSM_HsmSourceMoments_yy',
    135: 'ext_shapeHSM_HsmSourceMoments_xy',

    # This gives
    radius = (xx*yy - xy**2)**1/4

    # In the output  file we have
    # 1          2    34   56             78     9     10    11
    file_number, id,  x,y  xsigma,ysigma, e1,e2, ellip flux, radius
    """

    df = pd.read_csv(ifile, sep=",",low_memory=False)
    df.columns = df.columns.str.lstrip('# ')
    
    # make dtype float
    df = df.astype(float)

    # extra filtering  
    for num in lst_flag_nums:
        col_flag =  dict_flags_all[ str(num)]
        df = df[df[col_flag] == float(flag_TF) ]     

    # select only few columns
    usecols = [1, 94, 90, 102, 103, 104, 105, 127, 128, 114, 133,134,135]
    df = df.iloc[:,usecols]
    df = df.copy()

    # make selected columns numeric
    for c in df.columns:
        df[c] = pd.to_numeric(df[c],errors='coerce')


    # filter the flag calib_psfCandidate==False
    # not a star candidate
    df = df.query('calib_psfCandidate == 0.0')

    # filter the column deblend_nChild==0
    # no child source after deblending
    df = df.query('deblend_nChild == 0.0')

    # clean out unphysical results
    # e1^2 + e2^2 < 1.5^2
    df = df.copy()
    df['ellip'] = (df['ext_shapeHSM_HsmShapeRegauss_e1'] ** 2 +
                   df['ext_shapeHSM_HsmShapeRegauss_e2'] ** 2)**0.5
    df = df.query('ellip < 1.5')

    # calculate radius of ellipse using HSM moments
    # radius**4 = xx*yy - xy**2
    df['radius'] =  df.eval(""" ( (ext_shapeHSM_HsmSourceMoments_xx *  ext_shapeHSM_HsmSourceMoments_yy) \
                                              -  (ext_shapeHSM_HsmSourceMoments_xy**2 ) )**0.25 """)

    # add a new column with file_number
    df['file_number'] = file_number

    # take only required columns
    cols_select = ['file_number', 'id',
           'base_SdssCentroid_x', 'base_SdssCentroid_y',
           'base_SdssCentroid_xSigma','base_SdssCentroid_ySigma',
           'ext_shapeHSM_HsmShapeRegauss_e1','ext_shapeHSM_HsmShapeRegauss_e2',
           'ellip', 'base_SdssShape_flux',  'radius'
           ]

    df = df[cols_select]

    # drop all nans
    df = df.dropna()

    # write txt file with commented header
    prefix = ' '*2
    header_line = prefix.join(cols_select)

    # from: ../data/dmstack_csv/src_lsst_mono_z1.5_000.csv
    # to  : dmstack_txt/dmstack_txt_flag_0T/src_lsst_mono_z1.5_000.txt
    #       dmstack_txt/dmstack_txt_flag_0F/src_lsst_mono_z1.5_000.txt
    
    dmstack_txt = odir_txtT if flag_TF == 1 else odir_txtF
    ofile = ifile.replace('../data/dmstack_csv', dmstack_txt)
    ofile = ofile.replace('.csv', '.txt')
    np.savetxt(ofile,df.values,header=header_line,delimiter='\t')

def func1():
    infiles = ['../data/dmstack_csv/src_lsst_z1.5_{:03d}.csv'.format(i) for i in range(RANGE)]
    for ifile in infiles:
        file_number = int(ifile.rstrip('.csv').split('_')[-1])
        remove_nans(ifile, file_number, 0)
        remove_nans(ifile, file_number, 1)

def func2():
    infiles = ['../data/dmstack_csv/src_lsst90_z1.5_{:03d}.csv'.format(i) for i in range(RANGE)]
    for ifile in infiles:
        file_number = int(ifile.rstrip('.csv').split('_')[-1])
        remove_nans(ifile, file_number, 0)
        remove_nans(ifile, file_number, 1)

def func3():
    infiles = ['../data/dmstack_csv/src_lsst_mono_z1.5_{:03d}.csv'.format(i) for i in range(RANGE)]
    for ifile in infiles:
        file_number = int(ifile.rstrip('.csv').split('_')[-1])
        remove_nans(ifile, file_number, 0)
        remove_nans(ifile, file_number, 1)

def func4():
    infiles = ['../data/dmstack_csv/src_lsst_mono90_z1.5_{:03d}.csv'.format(i) for i in range(RANGE)]
    for ifile in infiles:
        file_number = int(ifile.rstrip('.csv').split('_')[-1])
        remove_nans(ifile, file_number, 0)
        remove_nans(ifile, file_number, 1)

if __name__ == '__main__':
    p1 = Process(target=func1)
    p1.start()

    p2 = Process(target=func2)
    p2.start()

    p3 = Process(target=func3)
    p3.start()

    p4 = Process(target=func4)
    p4.start()

    # join them all
    p1.join()
    p2.join()
    p3.join()
    p4.join()

Overwriting b01_remove_nans_dmstack.py


In [9]:
ofile = 'dmstack_txt/src_lsst_mono_z1.5_000.csv'
flag_01 = 0
ofile.replace('.csv', f'_flag{flag_01}.txt')

'dmstack_txt/src_lsst_mono_z1.5_000_flag0.txt'

# Create final_text.txt using IMCAT
```bash
bash b02_combine_four_txts_to_lc_catalog.sh

Created: final/final_000.cat
Created: final/final_099.cat

readcathead: file format error # we get this if we get error.

Date: Nov 7, 2019
I got redcathead error
Fix: I forgot to delete catalogs/*.cat and final/*.cat
```

In [115]:
df1 = pd.read_csv('../a18_nov1_2019/dmstack_txt/src_lsst90_z1.5_099.txt',sep=r'\t',skiprows=1,header=None,engine='python')

df1.head(2)

,0,1,2,3,4,5,6,7,8,9,10
0,99.0,37.0,589.8035,45.0719,0.1933,0.1974,0.5771,-0.6785,0.890734,1882.7064,3.447925
1,99.0,39.0,1336.9200,55.5085,0.0589,0.0847,-1.0599,-0.5366,1.187993,10106.4629,3.917561


In [116]:
df2 = pd.read_csv('dmstack_txt/src_lsst90_z1.5_099.txt',sep=r'\t',skiprows=1,header=None,engine='python')

df2.tail(2)

,0,1,2,3,4,5,6,7,8,9,10
1805,99.0,7643.0,993.2342,3218.5874,0.0264,0.0251,0.3388,1.0255,1.080017,19742.3352,4.268948
1806,99.0,7650.0,988.2575,3241.1116,0.1980,0.1320,0.8632,0.2452,0.897350,5303.3168,4.461064


# Plot gmsq histogram using final_text.txt

In [2]:
%%writefile b03_plot_gmsq.py
# Author: Bhishan Poudel
# Date:  Nov 7, 2019
# Command: py b03_plot_gmsq.py [0]

import scipy
import os,sys,json
import numpy as np
import pandas as pd
import seaborn as sns
sns.set(color_codes=True)

pd.set_option('display.max_columns',200)

import matplotlib.pyplot as plt

# global variables
dict_flags_all = json.load(open('dict_flags.json'))

# arguments
lst_flag_nums_str = sys.argv[1] # [0], [0,1] etc
lst_flag_nums = eval(lst_flag_nums_str)

ofile_whole = ['results/flag/flag_' + '_'.join([str(i) for i in lst_flag_nums])][0] 
ofile_whole = ofile_whole + '.png'

ofile_zoom = ['results/zoom/zoom_' + '_'.join([str(i) for i in lst_flag_nums])][0] 
ofile_zoom = ofile_zoom + '.png'

ofile_text = ['results/text/zoom_' + '_'.join([str(i) for i in lst_flag_nums])][0] 
ofile_text = ofile_text + '.txt'

if not os.path.isdir('results'):
    os.makedirs('results')


#===============================================================================
colnames = """fN[0][0]       fN[1][0]       fN[2][0]       fN[3][0]
id[0][0]       id[1][0]       id[2][0]       id[3][0]
x[0] x[1]     
errx[0][0]     errx[0][1]     errx[1][0]     errx[1][1]
errx[2][0]     errx[2][1]     errx[3][0]     errx[3][1]
g[0][0]        g[0][1]        g[1][0]        g[1][1] 
g[2][0]        g[2][1]        g[3][0]        g[3][1]
ellip[0][0]    ellip[1][0]    ellip[2][0]    ellip[3][0]
flux[0][0]     flux[1][0]     flux[2][0]     flux[3][0]
radius[0][0]   radius[1][0]   radius[2][0]   radius[3][0]
gm[0]          gm[1]          gc[0]          gc[1]
""".split()

colnames = [i.strip() for i in colnames]

#===============================================================================
# final_text is obtained from imcat after combining m,m9,l,l9 text files
df = pd.read_csv('final/final_text.txt',sep=r'\s+',
                 comment='#',header=None)

df.columns = colnames

#===============================================================================
# Find total flux, gm**2 and gc**2
df['flux'] = df['flux[0][0]'] + df['flux[1][0]'] + df['flux[2][0]'] + df['flux[3][0]']
df['gm_sq'] = df['gm[0]']**2 + df['gm[1]']**2
df['gc_sq'] = df['gc[0]']**2 + df['gc[1]']**2

#===============================================================================
a,b = 0.5, 1.1
df_bad = df.query(" @a < gm_sq < @b")


obj_all = 'all objects = {:,}'.format(len(df))
obj_bad = 'bad objects = {:,}'.format(len(df_bad))
per_bad = 'bad objects percentage = {:.4f}% '.format(len(df_bad)/len(df)*100)

with open(ofile_text,'w') as fo:
    line = obj_all + ' ' + obj_bad + ' ' + per_bad + '\n'
    fo.write(line)

# whole figure Histogram
#===============================================================================
fig, ax = plt.subplots(1,1,figsize=(12,12))

# hist + density
g = sns.distplot(df['gm_sq'], bins=60, ax=ax, norm_hist=True, kde=True)

# text
ax.text(0.6,0.4,obj_all)
ax.text(0.6,0.6,obj_bad)
ax.text(0.6,0.8,per_bad)

# labels
ax.set_xlabel('gm_sq')
ax.set_ylabel('arbitrary density unit')

# limits
ax.set_xlim(0,1.5)

# ticks
ax.set_xticks(np.arange(0,1.5,0.1))


# vr lines
ax.axvline(x=a,c='r',ls='--',label=f'gmsq = {a}')
ax.axvline(x=b,c='r',ls='--', label=f'gmsq = {b}')

# legends
ax.legend()
plt.suptitle('Histogram for gmsq')
plt.savefig(ofile_whole)
plt.close()

# zoom y-axis
#================================================
fig, ax = plt.subplots(1,1,figsize=(12,12))
df['gm_sq'].hist(ax=ax, label='gm_sq',bins=60)

# text
ax.text(0.6,4000,obj_all)
ax.text(0.6,4200,obj_bad)
ax.text(0.6,4400,per_bad)

# labels
ax.set_xlabel('gm_sq')
ax.set_ylabel('count')

# limits
ax.set_xlim(0,1.5)


# vr lines
ax.axvline(x=a,c='r',ls='--',label=f'gmsq = {a}')
ax.axvline(x=b,c='r',ls='--', label=f'gmsq = {b}')

# legends
ax.legend()
plt.suptitle('Histogram for gmsq')

# ticks
y_top = 10_000
ax.set_xlim(0.1,1.5)
ax.set_ylim(0,y_top)
ax.set_yticks(np.arange(0,y_top,200))
ax.set_xticks(np.arange(0,1.5,0.1))

for x in np.arange(0,y_top,200):
    ax.axhline(y=x,color='g',alpha=0.3)

plt.savefig(ofile_zoom)
plt.close()


Overwriting b03_plot_gmsq.py


In [252]:
# ! /Users/poudel/Library/Enthought/Canopy/edm/envs/deeplr/bin/python b03_plot_gmsq.py [0,1]

# Run Script

In [254]:
# !cat ../a18_nov1_2019/run.sh

In [1]:
%%bash

cat > run.sh << EOL
# Estimated time 
# Nov 7 2019:  [ 4h 46m 18s413 | Nov 08 12:34AM ]
for i in {0..89}                                                                
   do
        lst="[\$i]" &&
        echo "Estimated time: 3 min 45 seconds." &&
        echo "Cleaning old files: catalogs, final and dmstack_txt" &&
        rm -rf catalogs/*.cat final/* dmstack_txt/* &&

        # remove nans from dmstack csv and create txt
        echo "Removing nans" &&
        /Users/poudel/Library/Enthought/Canopy/edm/envs/deeplr/bin/python b01_remove_nans_dmstack.py \$lst &&

        # read 100*4 tab separated txt files, and create only 100 cats
        # and one big final final/final_text.txt using imcat
        echo "Creating cat files" &&
        bash b02_combine_four_txts_to_lc_catalog.sh &&

        # read final_text.txt and create gmsq histogram plot
        echo "Creating gmsq plot"
        /Users/poudel/Library/Enthought/Canopy/edm/envs/deeplr/bin/python b03_plot_gmsq.py \$lst                                                
done;

EOL

In [300]:
%%bash
cat run.sh

for i in {0..1}                                                                
   do
        lst="[$i]" &&
        echo "Estimated time: 3 min 45 seconds." &&
        echo "Cleaning old files: catalogs, final and dmstack_txt" &&
        rm -rf catalogs/*.cat final/* dmstack_txt/* &&

        # remove nans from dmstack csv and create txt
        echo "Removing nans" &&
        /Users/poudel/Library/Enthought/Canopy/edm/envs/deeplr/bin/python b01_remove_nans_dmstack.py $lst &&

        # read 100*4 tab separated txt files, and create only 100 cats
        # and one big final final/final_text.txt using imcat
        echo "Creating cat files" &&
        bash b02_combine_four_txts_to_lc_catalog.sh &&

        # read final_text.txt and create gmsq histogram plot
        echo "Creating gmsq plot"
        /Users/poudel/Library/Enthought/Canopy/edm/envs/deeplr/bin/python b03_plot_gmsq.py $lst                                                
done;



In [296]:
%%bash

for i in {0..10}                                                                
   do
      lst="[$i]"
      echo "$lst";                                                   
done;

[0]
[1]
[2]
[3]
[4]
[5]
[6]
[7]
[8]
[9]
[10]


In [13]:
lst_flag_nums = [0,1]
str_flag_nums = '_'.join([str(i) for i in a])


odir_txt0 = 'dmstack_txt/' + 'dmstack_txt_'+ str_flag_nums + 'T'
odir_txt0

'dmstack_txt/dmstack_txt_0_1T'

In [298]:
!bash test.sh

[0]
[1]
[2]
[3]
[4]
[5]
[6]
[7]
[8]
[9]
[10]


In [48]:
%%bash
nums="[0,1]"
nums="${nums:1:${#nums}-2}" # 0,1
nums=($(echo "$nums" | tr ',' '\n'))

# echo "${nums[0]}"

for num in "${nums[@]}"
do
    for t in T F
    do
        final="final/final_""$num""$t"
        echo "$final"
    done;
done;

final/final_0T
final/final_0F
final/final_1T
final/final_1F
